In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer

pd.options.display.max_columns = 30
%matplotlib inline

# Text-analys

Koden för den här anteckningsboken baseras mycket på Jonathan Somas [artikel om text-frekvenser](http://jonathansoma.com/lede/foundations/classes/text%20processing/tf-idf/).

Syftet är att visa hur man tar ord från en text och behandlar dem så att det är lätt att hitta mönster. Då kan man förstå hur samma sorts maskin-inlärningsalgoritmer kan användas för text som man använder för vanliga data-observationer, eller för bilder.

Livsmedel | Kolhydrater | Fett | Protein
--- | --- | --- | ---
Vitkål | 4,7 | 0,1 | 1,13
Sojabönor färska förvällda | 4,9 | 6,4 | 10,87
Fläskkorv fett ca 23 % rå | 4 | 22,7 | 8

I tabellen för livsmedel blir de olika proportionerna mellan näringsvärden som ett "mönster", där programmet inte behöver veta att 0,1 just är fetthalten för vitkål, utan bara att det är ett tal som förekommer med 4,7 och 1,13 för den observation vi kallar vitkål.

Sekvenserna [4,7 | 0,1 | 1,13], [4,9 | 6,4 | 10,87] och [4 | 22,7 | 8] kan kanske ses som olika "profiler". Den ena går ner i mitten, den andra kontinuerligt uppåt och den tredje upp i mitten. 

Skulle vi få in ännu en observation [1,2 | 2,3 | 3,5] skulle kanske programmet anta att den nya observationen var mest lik sojabönor, baserat på mönstret med de stigande värdena (eller att det var vitkål, baserat på de relativt låga värdena för alla tre faktorerna kolhydrater, fett och protein.

På ett liknande sätt kan man analysera bilder och få dem att bilda mönster, genom att varje pixel blir som en "dimension" (det som skulle motsvara ett värde för ett näringsvärde i tabellen här) och varje bild motsvarar då ett livsmedel. Likheter mellan bilder blir då som mönster i värdena för pixlarna i bilden.

Den enklaste analysmetoden kallas "bag of words" -- man tar helt enkelt varje mening och behandlar som en påse av ord. Det betyder bland annat att man struntar i ordningen mellan orden, och bryr sig bara om förekomsten av vissa ord i meningen.

En exempeltext, tagen från [en sida med lättlästa nyheter.](http://8sidor.se/varlden/2018/01/sydkorea-vill-ha-mote-med-nordkorea/)

In [9]:
texts = [
    "Länderna Sydkorea och Nordkorea har varit fiender länge.",
    "Nordkoreas ledare har hotat Sydkorea och andra länder med kärnvapen.",
    "Men nu ska ledarna i Sydkorea och Nordkorea börja att prata med varandra.",
    "Vi hoppas att kunna mötas och prata om många frågor.",
    "Vi vill ha bättre samarbete, säger Cho Myoung-Gyon i Sydkoreas regering.",
    "Samtalen ska bland annat handla om de olympiska spelen som börjar om några veckor i Sydkorea.",
    "Nordkoreas ledare säger att landet kanske vill skicka idrottare till OS."
]

Vi börjar med att segmentera och räkna orden.

Den inbyggda funktionen split fungerar för att segmentera meningarna i ord.

In [10]:
"Men nu ska ledarna i Sydkorea och Nordkorea börja att prata med varandra.".split()

['Men',
 'nu',
 'ska',
 'ledarna',
 'i',
 'Sydkorea',
 'och',
 'Nordkorea',
 'börja',
 'att',
 'prata',
 'med',
 'varandra.']

Om vi använder scikit-learn så gör den det här snyggare och lägger till en del finesser.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

In [12]:
# .fit_transfer TOKENIZES and COUNTS
X = count_vectorizer.fit_transform(texts)

Det direkta resultatet är svårt att använda som det är, men går att konvertera till en vanlig array

In [13]:
X

<7x57 sparse matrix of type '<class 'numpy.int64'>'
	with 76 stored elements in Compressed Sparse Row format>

In [14]:
X.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 1, 

In [15]:
pd.DataFrame(X.toarray())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1
5,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,...,0,1,1,0,1,1,1,0,0,0,0,0,1,0,0
6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1


De sju meningarna i exemplet får var sin rad, och de 57 olika orden får en kolumn. Om ordet förekommer i meningen sätter man antalet i rutan. 

Det här är ju ingen analys specifikt framtagen för text, så de olika orden blir "features" (motsvarande näringsvärden för livmedlen)

In [17]:
count_vectorizer.get_feature_names()

['andra',
 'annat',
 'att',
 'bland',
 'bättre',
 'börja',
 'börjar',
 'cho',
 'de',
 'fiender',
 'frågor',
 'gyon',
 'ha',
 'handla',
 'har',
 'hoppas',
 'hotat',
 'idrottare',
 'kanske',
 'kunna',
 'kärnvapen',
 'landet',
 'ledare',
 'ledarna',
 'länder',
 'länderna',
 'länge',
 'med',
 'men',
 'myoung',
 'många',
 'mötas',
 'nordkorea',
 'nordkoreas',
 'nu',
 'några',
 'och',
 'olympiska',
 'om',
 'os',
 'prata',
 'regering',
 'samarbete',
 'samtalen',
 'ska',
 'skicka',
 'som',
 'spelen',
 'sydkorea',
 'sydkoreas',
 'säger',
 'till',
 'varandra',
 'varit',
 'veckor',
 'vi',
 'vill']

In [18]:
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names())

,andra,annat,att,bland,bättre,börja,börjar,cho,de,fiender,frågor,gyon,ha,handla,har,...,samarbete,samtalen,ska,skicka,som,spelen,sydkorea,sydkoreas,säger,till,varandra,varit,veckor,vi,vill
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1
5,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,...,0,1,1,0,1,1,1,0,0,0,0,0,1,0,0
6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1


In [43]:
# We'll make a new vectorizer
stop = ['andra', 'annat', 'att', 'bland', 'de', 'ha', 'har', 'i', 'med', 'men', 'många', 'nu', 'och','om', 'ska', 'som','till', 'varandra', 'varit', 'vi', 'vill']
count_vectorizer = CountVectorizer(stop_words=stop)
X = count_vectorizer.fit_transform(texts)
print(count_vectorizer.get_feature_names())

['bättre', 'börja', 'börjar', 'cho', 'fiender', 'frågor', 'gyon', 'handla', 'hoppas', 'hotat', 'idrottare', 'kanske', 'kunna', 'kärnvapen', 'landet', 'ledare', 'ledarna', 'länder', 'länderna', 'länge', 'myoung', 'mötas', 'nordkorea', 'nordkoreas', 'några', 'olympiska', 'os', 'prata', 'regering', 'samarbete', 'samtalen', 'skicka', 'spelen', 'sydkorea', 'sydkoreas', 'säger', 'veckor']


In [44]:
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names())

,bättre,börja,börjar,cho,fiender,frågor,gyon,handla,hoppas,hotat,idrottare,kanske,kunna,kärnvapen,landet,...,nordkorea,nordkoreas,några,olympiska,os,prata,regering,samarbete,samtalen,skicka,spelen,sydkorea,sydkoreas,säger,veckor
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0
5,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1
6,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,...,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0


Kanske räcker det här för att man ska få en uppfattning om datastrukturen.

Sklearn har inte bra stöd för svenska, så jag tror att jag går över till gensim i stället. Där finns det också bra tutorials men de börjar inte på riktigt samma grundläggande nivå. [Här är nästa steg](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim%20Quick%20Start.ipynb).